In [1]:
from sapphire import *

    Management IP   Password
0   10.127.119.83   nbv_3124
1  10.127.119.103   nbv_3124
2   10.127.119.84   nbv_3124
3   10.127.119.98   nbv_3124
4  10.127.125.228  nbv_12345


In [2]:
mgmt_ip = '10.127.190.136'
password = 'nbv_12345'

mgmt_ip = '10.127.125.228'
password = 'nbv_12345'

mgmt_ip = '10.127.119.98'
password = 'nbv_3124'

mgmt_ip = '172.25.169.24'
password = 'nbv_12345'

mgmt_ip = '10.197.155.82'
password = 'n'




mgmt_ip = '10.127.190.134'

mgmt_ip = '10.127.190.138'

mgmt_ip = '10.127.125.219'


password = 'nbv_12345'

switch = Switch(mgmt_ip, password)

#vsan = int(input('Please enter the VSAN of interest : '))
vsan = 1000

Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fdfa08a44c0>
Switch object created with paramiko client in-built.


In [3]:
switch

In [4]:
fabric = Fabric(switch)

Fabric object initialized


In [5]:
fabric.search_devices(vsan = vsan)

PEER IP = 10.127.125.216 SWITCHNAME = N9k-core-DUT1
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fdfa083af70>
Switch object created with paramiko client in-built.
Device succeesfully added to Fabric.
PEER IP = 10.127.125.217 SWITCHNAME = Yushan-core-DUT1
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fdfa0910820>
Switch object created with paramiko client in-built.
Device succeesfully added to Fabric.
PEER IP = 10.127.125.218 SWITCHNAME = Yushan-core2-DUT1
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fdfa09315b0>
Switch object created with paramiko client in-built.
Device succeesfully added to Fabric.
PEER IP = 10.127.125.237 SWITCHNAME = N9k-StArch-Core1
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fdfa0

Switch object created with paramiko client in-built.
Device already present in the Fabric.
PEER IP = 10.127.125.236 SWITCHNAME = N9k-StArcher-DUT1
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fdfa0cb6d00>
Switch object created with paramiko client in-built.
Device already present in the Fabric.
Getting value error, hence skipping this iteration
PEER IP = 10.127.125.184 SWITCHNAME = Steller-Edge1
Please enter device password : nbv_12345
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fdf9cf603a0>
Switch object created with paramiko client in-built.
Device already present in the Fabric.
PEER IP = 10.127.125.184 SWITCHNAME = Steller-Edge1
Please enter device password : nbv_12345
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fdfa0c2adf0>
Switch object created with paramiko client in-built.
Device already present in the Fabric.
PEER IP = 10.127.125.216 SW

Switch object created with paramiko client in-built.
Device already present in the Fabric.
PEER IP = 10.127.125.216 SWITCHNAME = N9k-core-DUT1
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fdfa0973f70>
Switch object created with paramiko client in-built.
Device already present in the Fabric.
PEER IP = 10.127.125.217 SWITCHNAME = Yushan-core-DUT1
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fdfa09b1ca0>
Switch object created with paramiko client in-built.
Device already present in the Fabric.
PEER IP = 10.127.125.218 SWITCHNAME = Yushan-core2-DUT1
Password fetched from provided file.
Paramiko client created successfully :  <paramiko.client.SSHClient object at 0x7fdfa0973df0>
Switch object created with paramiko client in-built.
Device already present in the Fabric.
PEER IP = 10.127.125.183 SWITCHNAME = Steller_Core1
Please enter device password : 

In [6]:
for device in fabric.devices:
    device.print_details()
    print(device.show_flogi_database(vsan = vsan))

Switch Name :  N9k-Edge1-DUT1
Management IP : 10.127.125.219 Password : nbv_12345
Description :  "Nexus9000 C93180YC-FX Chassis"

  Interface  VSAN      FCID                PORT NAME                NODE NAME
0    fc1/31  1000  0x190160  50:06:01:64:49:e0:5b:95  50:06:01:60:c9:e0:5b:95
None
Switch Name :  N9k-core-DUT1
Management IP : 10.127.125.216 Password : nbv_12345
Description :  "Nexus9000 C93180YC-FX Chassis"

Empty DataFrame
Columns: [Interface, VSAN, FCID, PORT NAME, NODE NAME]
Index: []
None
Switch Name :  Yushan-core-DUT1
Management IP : 10.127.125.217 Password : nbv_12345
Description :  "MDS 9396T 96X32G FC (2 RU) Chassis"

Empty DataFrame
Columns: [Interface, VSAN, FCID, PORT NAME, NODE NAME]
Index: []
None
Switch Name :  Yushan-core2-DUT1
Management IP : 10.127.125.218 Password : nbv_12345
Description :  "MDS 9396T 96X32G FC (2 RU) Chassis"

Empty DataFrame
Columns: [Interface, VSAN, FCID, PORT NAME, NODE NAME]
Index: []
None
Switch Name :  N9k-StArch-Core1
Management IP :

In [7]:
switch.show_int_brief_fc()

Fetching fc interfaces...
fc1/1       1      E      on      trunking     swl   TE     32     103      

fc1/2       1      E      on      trunking     swl   TE     32     103      

fc1/3       1      E      on      trunking     swl   TE     32     103      

fc1/4       1      E      on      trunking     swl   TE     32     103      

fc1/5       1      E      on      trunking     swl   TE     32     103      

fc1/6       1      E      on      trunking     swl   TE     32     103      

fc1/7       1      E      on      trunking     swl   TE     32     103      

fc1/8       1      E      on      trunking     swl   TE     32     103      

fc1/9       1      E      on      sfpAbsent    --     --    --     --       

fc1/10      1      E      on      sfpAbsent    --     --    --     --       

fc1/11      1      E      on      sfpAbsent    --     --    --     --       

fc1/12      1      E      on      sfpAbsent    --     --    --     --       

fc1/13      1      E      on      notC

In [8]:
switch.show_flogi_database(vsan = vsan)

  Interface  VSAN      FCID                PORT NAME                NODE NAME
0    fc1/31  1000  0x190160  50:06:01:64:49:e0:5b:95  50:06:01:60:c9:e0:5b:95


In [9]:
switch.show_flogi_database(vsan = vsan)

  Interface  VSAN      FCID                PORT NAME                NODE NAME
0    fc1/31  1000  0x190160  50:06:01:64:49:e0:5b:95  50:06:01:60:c9:e0:5b:95


In [10]:
for key,value in switch.fc_interface.items():
    print(key, value)

fc1/1 <sapphire.FcInterface object at 0x7fdfa0ca8790>
fc1/2 <sapphire.FcInterface object at 0x7fdfa0973e80>
fc1/3 <sapphire.FcInterface object at 0x7fdfa0c98790>
fc1/4 <sapphire.FcInterface object at 0x7fdfa0c98970>
fc1/5 <sapphire.FcInterface object at 0x7fdfa0c989d0>
fc1/6 <sapphire.FcInterface object at 0x7fdf9cf91250>
fc1/7 <sapphire.FcInterface object at 0x7fdfa0e71be0>
fc1/8 <sapphire.FcInterface object at 0x7fdfa083a9d0>
fc1/9 <sapphire.FcInterface object at 0x7fdfa0c96100>
fc1/10 <sapphire.FcInterface object at 0x7fdfa0c37af0>
fc1/11 <sapphire.FcInterface object at 0x7fdfa098fb50>
fc1/12 <sapphire.FcInterface object at 0x7fdfa098f310>
fc1/13 <sapphire.FcInterface object at 0x7fdfa098fa00>
fc1/14 <sapphire.FcInterface object at 0x7fdfa098fa30>
fc1/15 <sapphire.FcInterface object at 0x7fdfa098f550>
fc1/16 <sapphire.FcInterface object at 0x7fdfa098f640>
fc1/17 <sapphire.FcInterface object at 0x7fdfa098fe20>
fc1/18 <sapphire.FcInterface object at 0x7fdfa098f070>
fc1/19 <sapphire.Fc

In [11]:
switch.fc_interface['fc1/1'].show()

client <paramiko.client.SSHClient object at 0x7fdfa08a44c0>
name fc1/1
vsan 1
admin_mode E
admin_trunk_mode on
status trunking
sfp swl
oper_mode TE
oper_speed 32
port_channel 103


In [12]:
switch.fc_interface['fc1/17'].__dict__

{'client': <paramiko.client.SSHClient at 0x7fdfa08a44c0>,
 'name': 'fc1/17',
 'vsan': '1',
 'admin_mode': 'E',
 'admin_trunk_mode': 'on',
 'status': 'trunking',
 'sfp': 'swl',
 'oper_mode': 'TE',
 'oper_speed': '32',
 'port_channel': '104'}

In [13]:
#switch.fc_interface['fc1/17'].flap()

In [14]:
switch.fc_interface['fc1/17'].show_brief()

<paramiko.client.SSHClient object at 0x7fdfa08a44c0> fc1/17 1 E on trunking swl TE 32 104 

In [15]:
switch.show_zoneset_active(vsan = vsan)

Getting active zoneset info for vsan 1000
Active Zoneset : Zoneset_SAN-A
zone_Baseport_CISCO_40G_SAN-A [('fcid', '0x230080'), ('fcid', '0x190160')]
zone_Baseport_Qlogic_HBA_32G_SAN-A [('fcid', '0x230000'), ('fcid', '0x190160')]
zone_Baseport_CISCO_100G_SAN-A [('fcid', '0x2300a0'), ('fcid', '0x190160')]
zone_Baseport_CISCO_25G_SAN-A [('fcid', '0x230060'), ('fcid', '0x190160')]
zone_Baseport_CISCO_VIC_10G_SAN-A [('fcid', '0x230040'), ('fcid', '0x190160')]
zone_Baseport_Emulex_HBA_16G_SAN-A [('fcid', '0x230020'), ('fcid', '0x190160')]


In [16]:
zone_dict = switch.get_zoneset_active(vsan = vsan)
print(zone_dict)

Getting active zoneset info for vsan 1000
Active Zoneset : Zoneset_SAN-A
{'zone_Baseport_CISCO_40G_SAN-A': [('fcid', '0x230080'), ('fcid', '0x190160')], 'zone_Baseport_Qlogic_HBA_32G_SAN-A': [('fcid', '0x230000'), ('fcid', '0x190160')], 'zone_Baseport_CISCO_100G_SAN-A': [('fcid', '0x2300a0'), ('fcid', '0x190160')], 'zone_Baseport_CISCO_25G_SAN-A': [('fcid', '0x230060'), ('fcid', '0x190160')], 'zone_Baseport_CISCO_VIC_10G_SAN-A': [('fcid', '0x230040'), ('fcid', '0x190160')], 'zone_Baseport_Emulex_HBA_16G_SAN-A': [('fcid', '0x230020'), ('fcid', '0x190160')]}


In [17]:
df = pd.DataFrame.from_dict(zone_dict, orient = 'index')
print(df.shape)
df.head(df.shape[0])

(6, 2)


,0,1
zone_Baseport_CISCO_40G_SAN-A,"(fcid, 0x230080)","(fcid, 0x190160)"
zone_Baseport_Qlogic_HBA_32G_SAN-A,"(fcid, 0x230000)","(fcid, 0x190160)"
zone_Baseport_CISCO_100G_SAN-A,"(fcid, 0x2300a0)","(fcid, 0x190160)"
zone_Baseport_CISCO_25G_SAN-A,"(fcid, 0x230060)","(fcid, 0x190160)"
zone_Baseport_CISCO_VIC_10G_SAN-A,"(fcid, 0x230040)","(fcid, 0x190160)"
zone_Baseport_Emulex_HBA_16G_SAN-A,"(fcid, 0x230020)","(fcid, 0x190160)"


In [18]:
switch.show_fcns_database(vsan = vsan)

Getting fcns database for vsan 1000
('0x190160', 'N', '50:06:01:64:49:e0:5b:95', 'Clariion', 'scsi', 'fcp', ['both'])
('0x230000', 'N', '21:00:f4:e9:d4:58:2a:64', None, 'scsi', 'fcp', ['init'])
('0x230020', 'N', '10:00:00:90:fa:f0:78:98', 'Emulex', 'scsi', 'fcp', ['init'])
('0x230040', 'N', '20:00:28:ac:9e:7c:78:45', 'Cisco', 'scsi', 'fcp', ['init', 'fc-gs'])
('0x230060', 'N', '20:00:3c:57:31:e5:2a:75', None, 'scsi', 'fcp', ['init', 'fc-gs'])
('0x230080', 'N', '20:00:d8:b1:90:18:e3:ea', 'Cisco', 'scsi', 'fcp', ['init', 'fc-gs'])
('0x2300a0', 'N', '20:00:a8:b4:56:50:4e:e2', None, 'scsi', 'fcp', ['init', 'fc-gs'])
('0x250000', 'N', '50:06:01:67:49:e0:5b:95', 'Clariion', 'scsi', 'fcp', ['both'])
('0x250020', 'N', '50:06:01:63:49:e0:5b:95', 'Clariion', 'scsi', 'fcp', ['both'])


In [19]:
switch.show_flogi_database(vsan = vsan)

  Interface  VSAN      FCID                PORT NAME                NODE NAME
0    fc1/31  1000  0x190160  50:06:01:64:49:e0:5b:95  50:06:01:60:c9:e0:5b:95


In [ ]:
# See https://neo4j.com/developer/aura-connect-driver/ for Aura specific connection URL.
scheme = "neo4j"  # Connecting to Aura, use the "neo4j+s" URI scheme
host_name = "localhost"
port = 7687
url = "{scheme}://{host_name}:{port}".format(scheme=scheme, host_name=host_name, port=port)
user = "admin"
password = "admin"
app = App(url, user, password)

In [ ]:
for switch in fabric.devices:
    switch.print_details()
    if 'MDS' in switch.descr:
        continue
    switch.show_int_brief_fc()
    for interface in switch.fc_interface:
        print ('Creating ports from {0}, interface : {1}'.format(switch.switchname, interface))
        app.create_port_node(switch.switchname, interface)
        
    switch.show_int_brief_vfc()
    for interface in switch.vfc_interface:
        print ('Creating ports from {0}, interface : {1}'.format(switch.switchname, interface))
        app.create_port_node(switch.switchname, interface)
        
    switch.show_int_brief_sanpo()
    for interface in switch.sanpo_interface:
        print ('Creating ports from {0}, interface : {1}'.format(switch.switchname, interface))
        app.create_port_node(switch.switchname, interface)

In [ ]:
# Not supporting E links currently
'''
for switch in fabric.devices:
    switchname = switch.switchname
    print(switch.switchname, switch.mgmt_ip)
    peers = switch.get_topology(vsan = 1000)
    print(peers.head())
    for i in range(peers.shape[0]):
        peer_switch = peers['Switch Name'][i]
        app.create_link(switchname, peer_switch, 'E')
'''


In [ ]:
for switch in fabric.devices:
    switchname = switch.switchname
    flogi_df = switch.get_flogi_database(vsan = vsan)
    print(flogi_df.head())
    for i in range(flogi_df.shape[0]):
        fcid = flogi_df['FCID'][i]
        port = flogi_df['Interface'][i]
        switch_port = switchname + '_' + port
        app.create_link(switch_port, fcid, 'N')

In [ ]:
from tqdm import tqdm 

# Getting zone config
zone_dict = switch.get_zoneset_active(vsan = vsan)
zone_df = pd.DataFrame.from_dict(zone_dict, orient = 'index')


for index, row in tqdm(zone_df.iterrows(), total = zone_df.shape[0]): 
    #print('Iterating over devices under zone {}'.format(index))
    concerned_ports = []
    for i in range(len(row)):
        if row[i] == None:
            #Ignoring empty cells
            continue
            
        # row[i][0] is the member type, and row[i][1] is the value.    
        switch_port = get_switch_port(fabric, vsan, row[i][0], row[i][1])
        if switch_port != None:
            concerned_ports.append(switch_port)
            
    #Now, create links between all the devices under the same zone.
    for i in range(len(concerned_ports)):
        for j in range(i+1, len(concerned_ports)):
            #print('Adding virtual link between {} and {}'.format(concerned_ports[i], concerned_ports[j]))
            app.create_link(concerned_ports[i], concerned_ports[j], 'V')

            

In [ ]:
app.close()